In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.contrib.keras.api.keras.layers import Dense, Input
from tensorflow.contrib.keras.api.keras.models import Model

In [3]:
N = 1000
D = 10
trX = np.random.random((N, D))
W = np.random.random((D,))
trY = np.dot(trX, W) + np.random.randn(N) * 0.1
print trX.shape
print W.shape
print trY.shape

(1000, 10)
(10,)
(1000,)


In [4]:
# reset graph (without ipython restart)
tf.reset_default_graph()
sess = tf.InteractiveSession()

# Define NN using TF + Keras

In [5]:
KX = Input(batch_shape=(None, D), name="x")  # same as tf.placeholder
KY = Input(batch_shape=(None,), name="y")  # same as tf.placeholder
y_model = Dense(1, name="fc1")(KX)  # we can use Keras layers

In [6]:
KX

<tf.Tensor 'x:0' shape=(?, 10) dtype=float32>

In [7]:
y_model

<tf.Tensor 'fc1/BiasAdd:0' shape=(?, 1) dtype=float32>

# Still a TF graph

In [11]:
y_model.graph.get_operations()[:10]

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'y' type=Placeholder>,
 <tf.Operation 'fc1/random_uniform/shape' type=Const>,
 <tf.Operation 'fc1/random_uniform/min' type=Const>,
 <tf.Operation 'fc1/random_uniform/max' type=Const>,
 <tf.Operation 'fc1/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'fc1/random_uniform/sub' type=Sub>,
 <tf.Operation 'fc1/random_uniform/mul' type=Mul>,
 <tf.Operation 'fc1/random_uniform' type=Add>,
 <tf.Operation 'fc1/kernel' type=VariableV2>]

# Train TF way

In [12]:
%%time
cost = tf.square(KY - y_model)
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

tf.global_variables_initializer().run()
for i in range(10):
    for (_x, _y) in zip(trX, trY):
        sess.run(train_op, feed_dict={KX: [_x], KY: [_y]})

CPU times: user 7.29 s, sys: 1.3 s, total: 8.58 s
Wall time: 5.95 s


In [13]:
sess.run(sess.graph.get_tensor_by_name("fc1/kernel:0"))

array([[ 0.95783073],
       [ 0.33056808],
       [ 0.48477054],
       [ 0.21332446],
       [ 0.33588055],
       [ 0.47077721],
       [ 0.75664514],
       [ 0.08198258],
       [ 0.56871861],
       [ 0.12137637]], dtype=float32)

In [14]:
sess.run(sess.graph.get_tensor_by_name("fc1/bias:0"))

array([ 0.03763441], dtype=float32)

In [15]:
print W

[ 0.95375342  0.34123218  0.48604434  0.21336514  0.34543623  0.47725094
  0.75417624  0.09432659  0.57886658  0.11371392]


# Train Keras way

In [16]:
model = Model(inputs=KX, outputs=y_model)
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, 10)                0         
_________________________________________________________________
fc1 (Dense)                  (None, 1)                 11        
Total params: 11.0
Trainable params: 11.0
Non-trainable params: 0.0
_________________________________________________________________


In [17]:
model.fit(trX, trY, batch_size=1, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 1s - loss: 0.0512 - mean_squared_error: 0.0512     1  22/1000 [..............................] - ETA: 3s - loss: 0.3731 - mean_squared_error: 0.3731 

In [18]:
model.get_weights()

[array([[ 0.9472751 ],
        [ 0.33176461],
        [ 0.48850054],
        [ 0.20892613],
        [ 0.34227699],
        [ 0.46168256],
        [ 0.76263154],
        [ 0.08223391],
        [ 0.54974163],
        [ 0.11366677]], dtype=float32), array([ 0.02557149], dtype=float32)]